In [16]:
from tensorflow import keras
from tensorflow.keras.models import load_model
import pickle
import pandas as pd

### 1. 재료 불러오기

In [17]:
loadedModel  = load_model("selloutcore.h5")

In [18]:
with open("prefunc.dump", "rb") as fr:
    preFunction = pickle.load(fr)

# 재훈련 (재훈련 아닌경우 바로 예측으로 점프)

In [19]:
gitUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/feature_regression_example.csv"

In [20]:
additionalData = pd.read_csv(gitUrl)

### 1.  타입 통합 및 특성 숫자컬럼 추가

In [21]:
requiredColumns = ["HOLIDAY","PROMOTION","HCLUS","PRO_PERCENT","QTY"]

In [22]:
addDataTrainSet = additionalData.loc [ :, requiredColumns ]

In [23]:
preProcessingFunc = preFunction[1]
ynLabelEncoder = preProcessingFunc[0]
hclusScaler = preProcessingFunc[2]
promoScaler = preProcessingFunc[3]

In [24]:
preFunction[0]

['LE_HOLY', 'LE_PROMO', 'SCALED_HCLUS', 'SCALED_PROPERCENT']

In [25]:
addDataTrainSet["LE_HOLY"] = ynLabelEncoder.fit_transform( addDataTrainSet.HOLIDAY )
addDataTrainSet["LE_PROMO"] = ynLabelEncoder.fit_transform( addDataTrainSet.PROMOTION )
addDataTrainSet["SCALED_HCLUS"] = hclusScaler.fit_transform( addDataTrainSet.loc[ :, ["HCLUS"]])
addDataTrainSet["SCALED_PROPERCENT"] = promoScaler.fit_transform( addDataTrainSet.loc[ :, ["PRO_PERCENT"]])

### 2. 특성선정 및 데이터 분리

In [26]:
features = preFunction[0]
label = ["QTY"]

In [27]:
trainDataFeatures = addDataTrainSet.loc[ :, features]
trainDataLabel = addDataTrainSet.loc[ :, label]

### 3. 모델 정의

### 4. 모델 컴파일

In [28]:
# 모델 컴파일
loadedModel.compile(loss="mean_squared_error",
                    optimizer="adam",
                    metrics= ["mean_absolute_error"])

### 5. 모델 훈련

In [29]:
loadedModel.fit(x=trainDataFeatures,
                y=trainDataLabel,
                batch_size=32,
                epochs=50,
                validation_split = 0.2)

Epoch 1/50
3/3 [==============================] - 1s 130ms/step - loss: 343412.5625 - mean_absolute_error: 415.9939 - val_loss: 358303.9062 - val_mean_absolute_error: 470.3341
Epoch 2/50
3/3 [==============================] - 0s 17ms/step - loss: 341157.2500 - mean_absolute_error: 416.4347 - val_loss: 354463.7500 - val_mean_absolute_error: 468.2234
Epoch 3/50
3/3 [==============================] - 0s 16ms/step - loss: 339967.0000 - mean_absolute_error: 416.5222 - val_loss: 350948.1875 - val_mean_absolute_error: 466.1362
Epoch 4/50
3/3 [==============================] - 0s 16ms/step - loss: 338885.5625 - mean_absolute_error: 417.0968 - val_loss: 348044.8438 - val_mean_absolute_error: 464.2645
Epoch 5/50
3/3 [==============================] - 0s 17ms/step - loss: 337984.3438 - mean_absolute_error: 418.1056 - val_loss: 345533.7812 - val_mean_absolute_error: 463.3278
Epoch 6/50
3/3 [==============================] - 0s 26ms/step - loss: 337028.2500 - mean_absolute_error: 418.1539 - val_los

# 예측

In [30]:
preProcessingFunc = preFunction[1]
ynLabelEncoder = preProcessingFunc[0]
hclusScaler = preProcessingFunc[2]
promoScaler = preProcessingFunc[3]

In [31]:
import warnings

In [32]:
warnings.filterwarnings(action = "ignore")

In [33]:
inputHoliday = "N"
inputPromotion ="N"
inputHclus = 4
inputPromoPercent = 0.0
inputHolidayModel = ynLabelEncoder.transform( [[inputHoliday]]) [0]
inputPromotionModel = ynLabelEncoder.transform( [[inputPromotion]]) [0]
inputHclusModel = hclusScaler.transform( [[inputHclus]] ) [0][0]
inputProPercentModel = promoScaler.transform( [[inputPromoPercent]] ) [0][0]
print(inputHolidayModel, inputPromotionModel, inputHclusModel, inputProPercentModel)

0 0 1.0 0.0


In [34]:
webData = pd.DataFrame( [[ inputHolidayModel ,inputPromotionModel,inputHclusModel, inputProPercentModel         ]  ]            )

In [35]:
loadedModel.predict(  webData  )

1/1 [==============================] - 0s 70ms/step


array([[298.77478]], dtype=float32)